<a href="https://colab.research.google.com/github/fromakim/2021Election_Analysis/blob/main/data_getfollowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Dev Environment

In [71]:
import itertools
import json
import requests
from time import sleep
from urllib import parse
from datetime import datetime, timedelta

In [2]:
import numpy as np
import pandas as pd

In [4]:
from google.colab import drive

In [5]:
drive.mount('/content/gdrive')
# LAMBDA: import boto3

Mounted at /content/gdrive


In [ ]:
# LABMDA: s3 = boto3.client('s3')

# 2. Key File Setting

In [6]:
f = open('./gdrive/MyDrive/Colab Notebooks/key.json')
# LAMBDA: f = open('./key.json')
keys = json.load(f)

In [7]:
api_key = keys['api_key']
api_secret_key = keys['api_secret_key']
bearer_token = keys['bearer_token']
app_id = keys['app_id']
index = int(keys['index'])

In [8]:
headers = {"Authorization" : f"Bearer {bearer_token}"}

# 3. Import Data

In [9]:
users = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/election_sample/user.csv')
# LAMBDA: users = pd.read_csv('./user.csv')

# 4. Collect Follow Network

In [ ]:
follows = pd.json_normalize(requests.get(f'https://api.twitter.com/2/users/{users["id"][index]}/following?max_results=1000', headers = headers).json()['data'])

In [ ]:
follows.columns = ['to', 'name', 'username']

In [ ]:
follows['from'] = users['id'][index]

In [ ]:
body = follows.to_json(orient = 'records')

'[{"to":"982126734","name":"\\u1d55\\u2240 \\u0320\\u1d55","username":"wouldujustnot","from":1236893975488778241},{"to":"977544986148274177","name":"MJ","username":"KIM_etal","from":1236893975488778241},{"to":"150633655","name":"Martin McKee","username":"martinmckee","from":1236893975488778241},{"to":"1181519339050156032","name":"June","username":"demipointe_139","from":1236893975488778241},{"to":"188244198","name":"jee","username":"limjees","from":1236893975488778241},{"to":"1209816676016873473","name":"\\uba74","username":"choimyeon","from":1236893975488778241},{"to":"275551267","name":"\\uae40\\uc5f0\\uacbd kim yeon koung","username":"kimyk10","from":1236893975488778241},{"to":"132762348","name":"JOY","username":"BF_cordwainer","from":1236893975488778241},{"to":"151764715","name":"\\uc0dd\\uac15","username":"inkivaariyksi","from":1236893975488778241},{"to":"43509988","name":"tablo","username":"blobyblo","from":1236893975488778241},{"to":"2972799367","name":"\\uc815\\ubc00\\uc544Jeon

In [ ]:
# LAMBDA: s3.put_object(Body = body, Bucket = 'electiondatafile', Key = f'follows/{users["id"][index]}.json')

# 5. Modify Key Settings

In [ ]:
keys['index'] = str(int(keys['index']) + 1)

In [ ]:
# LAMBDA: s3.put_object(Body = json.dumps(keys), Bucket = 'electiondatafile', Key = 'key.json')

Loop

In [59]:
users.columns = ['name', 'verified', 'created_at', 'description', 'profile_image_url', 'username', 'id', 'url', 'protected', 'location', 'pinned_tweet_id']

In [68]:
follows = pd.DataFrame(columns = ['to_id', 'to_name', 'to_username', 'from_id', 'from_name', 'from_username'])

In [127]:
for index, user in users[234:].iterrows():      # 바로 실행하면 됨.
    try:
        print(f'user analysis on index [{index}]')

        res = requests.get(f'https://api.twitter.com/2/users/{user.id}/following?max_results=1000', headers = headers).json()

        data = res['data']
        count = res['meta']['result_count']
        
        df = pd.json_normalize(data)
        df = df[['id', 'name', 'username']]
        df.columns = ['to_id', 'to_name', 'to_username']
        
        df['from_id'] = user['id']
        df['from_name'] = user['name']
        df['from_username'] = user['username']

        follows = pd.concat([follows, df])

    except:
        print(f'execution stopped at the index : [{index}]')
        print(res)

        # if res['errors'][0]['title'] == 'Authorization Error':
        #     sleep(90)
        #     continue
        continue

    finally:
        sleep(90)

user analysis on index [171]
user analysis on index [172]
user analysis on index [173]
user analysis on index [174]
user analysis on index [175]
user analysis on index [176]
execution stopped at the index : [176]
{'errors': [{'value': '1438745955327569927', 'detail': 'Could not find user with id: [1438745955327569927].', 'title': 'Not Found Error', 'resource_type': 'user', 'parameter': 'id', 'resource_id': '1438745955327569927', 'type': 'https://api.twitter.com/2/problems/resource-not-found'}]}
user analysis on index [177]
user analysis on index [178]
user analysis on index [179]
user analysis on index [180]
user analysis on index [181]
user analysis on index [182]
user analysis on index [183]
user analysis on index [184]
user analysis on index [185]
user analysis on index [186]
user analysis on index [187]
user analysis on index [188]
user analysis on index [189]
user analysis on index [190]
execution stopped at the index : [190]
{'errors': [{'value': '976145981661003777', 'detail': '

KeyboardInterrupt: ignored

In [128]:
follows

,to_id,to_name,to_username,from_id,from_name,from_username
0,982126734,ᵕ≀ ̠ᵕ,wouldujustnot,1236893975488778241,크즈,unkogej
1,977544986148274177,MJ,KIM_etal,1236893975488778241,크즈,unkogej
2,150633655,Martin McKee,martinmckee,1236893975488778241,크즈,unkogej
3,1181519339050156032,June,demipointe_139,1236893975488778241,크즈,unkogej
4,188244198,jee,limjees,1236893975488778241,크즈,unkogej
...,...,...,...,...,...,...
343,166739404,Emma Watson,EmmaWatson,794551904726433792,니니,puffinpeng
344,98567132,페미니스트 저널 일다,ildaro,794551904726433792,니니,puffinpeng
345,703107382625198080,기독교 덕후,christ_taku,794551904726433792,니니,puffinpeng
346,151807455,djuna,djuna01,794551904726433792,니니,puffinpeng


In [129]:
users[230:235]

,name,verified,created_at,description,profile_image_url,username,id,url,protected,location,pinned_tweet_id
230,카르텔v2.0,False,2017-04-18T12:26:09.000Z,구독계 플텍계 기타 내멋대로 블언블. 구독계라 의심되도 블언블 또는 차단\n\n사회...,https://pbs.twimg.com/profile_images/145652695...,_jbdjljm__,854310076546404352,NaN,False,NaN,1.454243e+18
231,#사사오입 #부정선거,False,2012-03-15T04:20:28.000Z,#효력정지_가처분신청_기각_판사의불법절차\n#일본제_보이콧\n#낙지털보사절\n#토착...,https://pbs.twimg.com/profile_images/144964187...,mujungyu,525022124,NaN,False,서울,NaN
232,👹TV조선홍회장vs전과4범,False,2010-05-30T12:27:42.000Z,공수처판사1호❤서울남부법원판사\n#판사남편_법적처벌안받는_불사신\n★662일째(20...,https://pbs.twimg.com/profile_images/125412334...,inin12340,149861557,NaN,False,NaN,1.437577e+18
233,니니,False,2016-11-04T14:48:30.000Z,볼수록 황홀한 넌 누구야 ❤️‍🔥,https://pbs.twimg.com/profile_images/102797005...,puffinpeng,794551904726433792,NaN,False,"New York, NY",1.438281e+18
234,심상정,True,2009-03-31T05:29:28.000Z,#심블리 가 여러분의 자부심이 되겠습니다. 정의당 국회의원 심상정입니다.\n\n문의...,https://pbs.twimg.com/profile_images/133506349...,sangjungsim,27821557,https://t.co/739nWIVSlo,False,Korea,NaN


In [117]:
follows.reset_index(drop = True).to_csv('followmap.csv')

In [116]:
follows

,to_id,to_name,to_username,from_id,from_name,from_username
0,982126734,ᵕ≀ ̠ᵕ,wouldujustnot,1236893975488778241,크즈,unkogej
1,977544986148274177,MJ,KIM_etal,1236893975488778241,크즈,unkogej
2,150633655,Martin McKee,martinmckee,1236893975488778241,크즈,unkogej
3,1181519339050156032,June,demipointe_139,1236893975488778241,크즈,unkogej
4,188244198,jee,limjees,1236893975488778241,크즈,unkogej
...,...,...,...,...,...,...
104,1445600122427957253,혜견,G_211006,1445587101790007302,냠,ddinni_
105,1233190331761381377,🤍말랑공주❤,MeEl10_,1445587101790007302,냠,ddinni_
106,1232912988459290624,클튜출석해💢무해,MUHE_FF14,1445587101790007302,냠,ddinni_
107,724774289765896192,루토스,Rutos_0814,1445587101790007302,냠,ddinni_
